# Demo - Linear Weights and wOBA

In the demo on RE24, we directly attributed RE24 to a player by computing the total RE24 the player produced over the season.  Instead, we can try to even out their perfomance by weighting each event the same by completely ignoring the context of the event.  This allows us to avoid the pitfalls we saw previously where it seemed nearly impossible to tell whether environment or ability was driving production.  By doing this, we completely remove environment in which the hitter was hitting in and get a performance compared to overall average!

We'll use the RE24 values to construct the _Linear Weights_, or _LWTS_, values.  For each `Event_Type` we can compute the average value for each type of event. We use then show how LWTS forms the basis for the advanced statistic _wOBA_ (Weighted On-Base Average) we saw in the offensive metrics demo.

A single can have different RE24 outcomes according to different out/baserunner situations so the average weights these outcome values according to the frequency with which the situations occur.  This way, we do not under value a single that occurs with no runners on and doesn't score a run or over value a single that occurs with the bases loaded.  We want them to be valued equally so we average to smooth everything out.


## Setup

In [ ]:
%run ../../utils/notebook_setup.py

In [ ]:
from datascience import Table, are
import numpy as np

# custom functions that will help do some simple tasks
from datascience_utils import *
from datascience_stats import *

## Load Data

We need the RE24 values for each event in the play-by-play retrosheet dataset.  We computed these values in the last demo so we can load the saved results.  

In [ ]:
retro = Table.read_table('retrosheet_with_re24-2017.csv.gz', sep=',')

In [ ]:
retro.show(10)

## 1. RE24 Values

We know that an plate appearance event, say a single, can have different RE24 values depending on the situation. What do all those various values look like though? 

### A. RE24 Histogram for a Single

We start with a single.  Most of the time the single produces run values under .5.  Occasionally it can produce run values between .5 and 1, while much rarer will it produce a run value well above 1 or even negative.

In [ ]:
bins = np.arange(-1, 2, .1)

# Histogram of 1B RE24 values
retro.select('Event_Type', 'RE24').\
    where('Event_Type', '1B').\
    hist('RE24', bins=bins)

### B. RE24 Histogram for a Homerun

We know a homerun scores at least 1 run and clears the bases.  Clearing the bases naturallty reduces the run expectancy but unless something insane happens in our dataset, every homerun should have an RE24 value $\geq 1$.



In [ ]:
bins = np.arange(1., 4, .1)

# Histogram of HR RE24 values
retro.select('Event_Type', 'RE24').\
    where('Event_Type', 'HR').\
    hist('RE24', bins=bins)

### C. Generic Outs vs Strikeouts

A generic out is when a batter makes contact with the ball and is put out by the defense.  A strikeout is when a batter fails to make a contact (or doesn't swing) and is out on 3 strikes.

Our intuition would say the generic out is better than a strikeout because you are putting the ball in play so positive outcomes could still occur (ie. runner advancement).  

In [ ]:
bins = np.arange(-2., 2, .1)

# Histogram of Out RE24 values
retro.select('Event_Type', 'RE24').\
    where('Event_Type', 'Generic out').\
    hist('RE24', bins=bins)

Putting the ball in produces errors and other events but in terms of outs, there is little difference to a strikeout.

In [ ]:
# Histogram of K RE24 values
retro.select('Event_Type', 'RE24').\
    where('Event_Type', 'K').\
    hist('RE24', bins=bins)

## 2. Event Run Values

We compute the weights for each event by averaging its RE24 values:
$$
    \text{Linear Weight for Event Type $E$} = \frac{1}{\text{# of Events $E$}}
        \sum \text{RE24 for $i$-th Event $E$}
$$

These weights form the run values for the events in _Linear Weights_, often abbreviated _LWTS_.

In [ ]:
# Group by event type and average
lwts = retro.select('Event_Type', 'RE24').\
    group('Event_Type', np.mean)
lwts.relabel('RE24 mean', 'RE24')
lwts.sort('RE24').show()

_Questions_

1. What units are the weight values in?
2. What does the weight value represent?  
3. Why do events that produce outs have negative values and events that produce baserunners have positive values?
4. Relatedly, what would a value of 0 mean?

## 3. wRAA/Linear Weights

Now that we have weights for each event, we can compute a statistic called _Weighted Runs Above Average_, or _wRAA_.  This statistic sometimes is also known as _Batting Runs_, which was developed by Pete Palmer.  Fangraphs uses the name wRAA.  This is also known as _Linear Weights_ or _LWTS_.

wRAA is computed by using the season totals for a player and summing the contributions.

In [ ]:
def get_weight(lwts, event):
    """Extract the linear weight for event"""
    return lwts.where('Event_Type', event)['RE24'].item()

In [ ]:
get_weight(lwts, 'K')

### A. Load 2017 Player Data

We load the 2017 player data from the Lahman dataset.

In [ ]:
players = Table.read_table('players_2017.csv', sep=',')

# Compute plate appearances and 1B
players['PA'] = players['AB'] + players['HBP'] + players['IBB'] + players['BB'] + players['SH'] + players['SF']
players['1B'] = players['H'] - players['2B'] - players['3B'] - players['3B'] - players['HR']
# Compute generic outs, approximately. We can't handle RBOE with the Lahman data
players['O'] = players['AB'] - players['H'] - players['SO']

In [ ]:
players.show(5)

### B. Compute wRAA

We use the weight values and the season counts for each player to compute their wRAA value.

In [ ]:
players['wRAA'] = \
    get_weight(lwts, 'Generic out') * players['O'] + \
    get_weight(lwts, 'K') * players['SO'] + \
    get_weight(lwts, 'IBB') * players['IBB'] + \
    get_weight(lwts, 'BB') * players['BB'] + \
    get_weight(lwts, 'HBP') * players['HBP'] + \
    get_weight(lwts, '1B') * players['1B'] + \
    get_weight(lwts, '2B') * players['2B'] + \
    get_weight(lwts, '3B') * players['3B'] + \
    get_weight(lwts, 'HR') * players['HR']

In [ ]:
players.sort('wRAA', descending=True).show(15)

_Questions_
1. What are the units for wRAA?
2. What does a value of 0 mean?  Is 0 bad?
3. Is wRAA a count or rate statistic?

### C. wRAA vs Total RE24

We saw that RE24 is a way to meaure total run production by a player.  wRAA is another way to measure total run production, but because it relies on the Linear Weights it is context/situation independent.

We recompute total RE24 for batters and we'll compare it to wRAA.

In [ ]:
# Collect totals by batter
batter_re24 = retro.select('Batter_ID', 'Run_Expectancy', 'RE24').\
    group('Batter_ID', collect=sum)
batter_re24.relabel(['Run_Expectancy sum', 'RE24 sum'], ['Run_Expectancy', 'RE24'])
# Sort and display the top 10
batter_re24.sort('RE24', descending=True).\
    show(10)

#### Joining wRAA and RE24

Unfortunately, the Lahman dataset and the Retrosheet dataset use different identifier tags for players.  Fortunately, a master table is available online to link the identifier tags to a player.  We'll use that master table now.

In [ ]:
# Load the master table for linking identifier tags from different sites
master = Table.read_table('master.csv')
master.show(5)

In [ ]:
# Join the RE24 and wRAA results
# 1. Join master to players on the Lahman ID
# 2. Join the result from 1 with the RE24 table on the Retrosheet ID
# 3. Subset the columns we care about
batter_data = master.join('lahman_id', players, other_label='playerID').\
    join('retro_id', batter_re24, other_label='Batter_ID').\
    select('mlb_name', 'PA', 'RBI', 'Run_Expectancy', 'RE24', 'wRAA')

batter_data.sort('wRAA', descending=True).show(15)

#### RE24 vs wRAA

We can generate a scatter plot to compare the two production measures.  If one looks really closely, one can potentially guess that wRAA appears to estimate production higher than RE24.  We should be hesitant to read too much into this for now: we have been computing wRAA via the Lahman data, which is known to not be totally comprehensive.  

In [ ]:
batter_data.scatter('RE24', 'wRAA')

## 4. wOBA

Recall wOBA, which is one of the premier advanced stats out there.  

The formula for wOBA is given by,
$$
    \mathit{wOBA} =
    \frac{0.72\cdot \mathit{BB} + 0.75\cdot \mathit{HBP} + 0.90\cdot \mathit{1B} + 1.24\cdot\mathit{2B} + 1.56\cdot\mathit{3B} + 1.95\cdot\mathit{HR}}{\mathit{PA}}
$$

So how do we derive the wOBA weights?  First, we should notice that wOBA does not penalize outs.  Like BA, OBP, and SLG, wOBA sets the value of an out at 0 and weights events relative to that.  

### A. Value Relative to an Out

We subtract the value of a generic out to determine the relative run value of an event relative to an out.

In [ ]:
rel_weight_BB = get_weight(lwts, 'BB') - get_weight(lwts, 'Generic out')
rel_weight_HBP = get_weight(lwts, 'HBP') - get_weight(lwts, 'Generic out')
rel_weight_1B = get_weight(lwts, '1B') - get_weight(lwts, 'Generic out')
rel_weight_2B = get_weight(lwts, '2B') - get_weight(lwts, 'Generic out')
rel_weight_3B = get_weight(lwts, '3B') - get_weight(lwts, 'Generic out')
rel_weight_HR = get_weight(lwts, 'HR') - get_weight(lwts, 'Generic out')

In [ ]:
# Print the relative weights
print(f"BB:  {rel_weight_BB:.3f}")
print(f"HBP: {rel_weight_HBP:.3f}")
print(f"1B:  {rel_weight_1B:.3f}")
print(f"2B:  {rel_weight_2B:.3f}")
print(f"3B:  {rel_weight_3B:.3f}")
print(f"HR:  {rel_weight_HR:.3f}")

### B. wOBA Scale

The wOBA weights for 2017 on Fangraphs (https://www.fangraphs.com/guts.aspx?type=cn) are


| Event | BB | HBP | 1B |  2B |  3B |  HR |
| ----- |----|-----|----|-----|-----|-----|
| Value |.693|.723 |.877|1.232|1.552|1.980|
	
So why are our values so different?  The wOBA Scale.

First, we should note that,
1. The linear weights are run values above average.  If you produce a 0 wRAA, you produced runs at the average rate.  
2. Subtracting the value of the out produces a relative value.  This relative value tells you how much your wRAA would increase if you converted one of your outs to another event.

So wRAA and the relative values should all make sense, but they're just not that informative for the general public. 

In order to facilitate an appreciation for wOBA, we can adjust wOBA so that it looks and feels similar to OBP but is actually better.  How do we do that?  We scale the weights so that league average wOBA is equal to league average OBP (typically around .320 or so).  Adjusting like this does nothing to the power of wOBA, it just changes it so that a player with a wOBA of .320 is around average.

In [ ]:
# wOBA scale for 2017 from FanGraphs
woba_scale = 1.185

Using the wOBA scale from FanGraphs, we get something that is basically identical.  We're unable to replicate their work indentically because of many individual, intricate steps but we're close enough.

In [ ]:
# Print the wOBA weights
print(f"BB:  {woba_scale * rel_weight_BB:.3f}")
print(f"HBP: {woba_scale * rel_weight_HBP:.3f}")
print(f"1B:  {woba_scale * rel_weight_1B:.3f}")
print(f"2B:  {woba_scale * rel_weight_2B:.3f}")
print(f"3B:  {woba_scale * rel_weight_3B:.3f}")
print(f"HR:  {woba_scale * rel_weight_HR:.3f}")